In [1]:
#default_exp prep_novasar

# Prep NovaSAR

> Data preparation for NovaSAR acquisitions. SLC to dB for Gamma0.

In [4]:
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
#hide
from nbdev.showdoc import *

In [15]:
#export
import os
import time
from glob import glob
import shutil

from ard_workflows.prep_utils import sedas_client, sedas_download, sedas_extract
from ard_workflows.prep_utils import run_snap_command, setup_logging
from ard_workflows.prep_utils import convert2cog, s3_upload_dir

In [7]:
import pandas as pd

notes
- default dem of auto-downloading srtm 1-arc sec
- ML+TC+TF default (and currently only) workflow
- gamma0
- explode sub-scenes to scenes
- no compression on cog
- copy original metadata, but not outmetadata

In [18]:
#export
def prep_novasar(sedas_supplierId, inter_dir="/tmp/data/intermediate/", 
                  s3_bucket="public-eo-data", s3_dir="novasar_uk_test/"):
    """
    Prepare a Sedas-hosted NovaSAR .zip scene(/strip) by suppliedId into a flavour of ARD
    using SNAP and upload output COGs to Catapult object storage. 
    """
    try:
        snap_gpt = os.environ['SNAP_GPT']
        novasar_process = os.environ['NOVASAR_PROCESS']
        root = setup_logging()

        inter_dir = f"{inter_dir}{sedas_supplierId}_tmp/"
        os.makedirs(inter_dir, exist_ok=True)
        master_scene_name = sedas_supplierId
        down_zip = f"{inter_dir}{master_scene_name}.zip"
        master_scene_dir = f"{down_zip[:-4]}/"
        
        
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Preparing {master_scene_name} within {inter_dir}")
        # find & download
        sedas_scene_res = sedas_client().search_product(sedas_supplierId)[0]
        sedas_download([sedas_scene_res], inter_dir)
        sedas_extract(down_zip, master_scene_dir)

        scenes_metadata = glob(f"{inter_dir}**/*metadata.xml", recursive=True)
        
        for scene_metadata in scenes_metadata:
            
            scene_name = scene_metadata.split('/')[-2]
            print(scene_name)
            
            out_dim = f"{inter_dir}{scene_name}_ML_TC_TF.dim"
            out_dimdir = f"{out_dim[:-4]}.data/"
            print(out_dimdir)
            
            out_cog_dir = f"{out_dimdir.replace('.data','_cog')}"
            os.makedirs(out_cog_dir, exist_ok=True)
            print(out_cog_dir)
            
            cmd = [snap_gpt, novasar_process, f"-Pinput={scene_metadata}", f"-Poutput={out_dim}"]
            run_snap_command(cmd)
            
            scene_bands = glob(f"{out_dimdir}**/*.img", recursive=True)
            for band_path in scene_bands:
                out_cog = f"{out_cog_dir}{scene_name}_{os.path.basename(band_path)[:-4]}.tif"
                print(out_cog)
                convert2cog(band_path, out_cog, 1)
            
            shutil.copy(scene_metadata, f"{out_cog_dir}original_metadata.xml")
            shutil.copy(out_dim, f"{out_cog_dir}process_metadata.dim")
            
            # upload
            s3_upload_dir(out_cog_dir, s3_bucket, s3_dir)
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Prepared {master_scene_name} at {s3_dir}")
#         clean_up(inter_dir) 
    except Exception as e:
        print(f"{time.strftime('%Y-%m-%d %H:%M:%S')} Failed with {e}")    
#         clean_up(inter_dir)

In [17]:
prep_novasar('NovaSAR_01_14295_slc_200823_233848_HH')

DEBUG:sedas_api:successful login.


2020-11-17 16:25:03 Preparing NovaSAR_01_14295_slc_200823_233848_HH within /tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/


DEBUG:sedas_api:successful login.
DEBUG:sedas_api:successful login.
DEBUG:sedas_api:successful login.
DEBUG:sedas_api:successful login.
INFO:bulk_downloader:0 downloads pending, 0 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.


2020-11-17 16:25:04 Downloading


DEBUG:bulk_downloader:1 downloads pending
DEBUG:bulk_downloader:1 downloads pending
INFO:bulk_downloader:downloading NovaSAR_01_14295_slc_200823_233848_HH to /tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_200823_233848_HH.zip
DEBUG:sedas_api:successful login.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:bulk_downloader:0 downloads pending, 1 downloads in progress, 0 requests pending
INFO:bulk_downloader:2 active threads, 0 restarted.
INFO:b

...

/tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_11_200823_233912_HH_7_ML_TC_TF_cog/NovaSAR_01_14295_slc_11_200823_233912_HH_7_Intensity_HH_db.tif
2020-11-17 16:34:16 Starting conversion: /tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_11_200823_233912_HH_7_ML_TC_TF.data/Intensity_HH_db.img.
2020-11-17 16:34:22 Conversion complete: True.


DEBUG:root:running ['/opt/snap/bin/gpt', '/opt/notebooks/anc/novasar_slc_tc.xml', '-Pinput=/tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_200823_233848_HH/SV229/Image_Products$/NovaSAR_01_14295_slc_200823_233848_HH_/NovaSAR_01_14295_slc_11_200823_233916_HH_8/metadata.xml', '-Poutput=/tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_11_200823_233916_HH_8_ML_TC_TF.dim']


NovaSAR_01_14295_slc_11_200823_233916_HH_8
/tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_11_200823_233916_HH_8_ML_TC_TF.data/
/tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_11_200823_233916_HH_8_ML_TC_TF_cog/


INFO:snap_stderr:stderr:INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO:snap_stderr:stderr:INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Incompatible GDAL 3.1.4 found on system. Internal GDAL 3.0.0 from distribution will be used.
INFO:snap_stderr:stderr:INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO:snap_stderr:stderr:INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
INFO:snap_stderr:stderr:INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO:snap_stdout:Executing processing graph
INFO:snap_stderr:stderr:INFO: org.hsqldb.persist.Logger: dataFileCache open start
INFO:snap_stderr:stderr:INFO: org.esa.snap.core.dataop.dem.ElevationFile: http retrieving http://step.esa.int/auxdata/dem/SRTMGL1/N50W003.SRTMGL1.hgt.zip
INFO:snap_stdout:....10%....20%....30%....40%....50%....

/tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_11_200823_233916_HH_8_ML_TC_TF_cog/NovaSAR_01_14295_slc_11_200823_233916_HH_8_Intensity_HH_db.tif
2020-11-17 16:35:27 Starting conversion: /tmp/data/intermediate_test/NovaSAR_01_14295_slc_200823_233848_HH_tmp/NovaSAR_01_14295_slc_11_200823_233916_HH_8_ML_TC_TF.data/Intensity_HH_db.img.
2020-11-17 16:35:39 Conversion complete: True.
2020-11-17 16:36:47 Prepared NovaSAR_01_14295_slc_200823_233848_HH at novasar_uk_test/



## Export

In [19]:
#hide
from nbdev.export import notebook2script; notebook2script()

Converted 1_prep_utils.ipynb.
Converted 2_prep_novasar.ipynb.
